In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

from robot.ml.custom_transformers import TextCleaner, DomainBinarizer
from robot.models import AccountType

In [5]:
# Создание DataFrame
file_path = 'data/modеls/train_data/merged_table.xlsx'
df = pd.read_excel(file_path)

df.head()

,Ссылка на аккаунт,Описание страницы,Найденная почта,Ссылки из описания,Ссылки из контактов,Кол-во постов,Кол-во подписчиков,Кол-во подписок,"Хэштег, по которому найден аккаунт",Дата сохранения,Предсказанный тип аккаунта,Комментарий верификатора,"Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER"
0,https://www.instagram.com/wifimoneymotivated/,Wifi Money ™️\nПредприниматель\nWealth | Motiv...,NaN,NaN,NaN,151,1975,124,#upcomingrapper,2024-12-12 23:50:14.547900,OTHER,NaN,NaN
1,https://www.instagram.com/floridadigitalrecords/,Tampaworldstar 🧟‍♂️\nСтудия звукозаписи\nYour ...,NaN,https://www.instagram.com/tampaworldstarofficial/,NaN,2961,19400,18,#upcomingrapper,2024-12-12 23:50:14.565188,LABEL,NaN,NaN
2,https://www.instagram.com/tampaworldstarofficial/,Worldstartampa 🧟‍♂️\nСтудия звукозаписи\nYour ...,NaN,https://www.instagram.com/worldstarfloridaoffi...,NaN,4772,21600,302,#upcomingrapper,2024-12-12 23:50:14.575259,LABEL,NaN,NaN
3,https://www.instagram.com/worldstarfloridaoffi...,Tampaworldstar 🧟‍♂️\nСтудия звукозаписи\nYour ...,NaN,https://www.instagram.com/floridadigitalrecord...,NaN,8476,115000,6547,#upcomingrapper,2024-12-12 23:50:14.585231,LABEL,NaN,NaN
4,https://www.instagram.com/zentheproducer/,Zentheproducer\nzentheproducer\n*Multi-Genre M...,NaN,https://www.instagram.com/theproducerzen/\nhtt...,NaN,2530,13700,6112,#upcomingrapper,2024-12-12 23:50:15.923423,BEATMAKER,NaN,NaN


In [34]:
##################################
# Шаг 1: Предобработка текста
##################################

# Создание DataFrame
file_path = 'data/modеls/train_data/merged_table.xlsx'
df = pd.read_excel(file_path)

# Замена пустых значений
df.fillna(
    {   
        'Предсказанный тип аккаунта': 'OTHER',
        'Описание страницы': '',
        'Ссылки из описания': '', 
        'Ссылки из контактов': '',
        'Кол-во постов': 0
    }, 
    inplace=True
)

# 1 шаг: Замена имени столбца
# 2 шаг: Удаление всех строк с типом "OTHER"
df.rename(columns={"Предсказанный тип аккаунта": "Тип аккаунта"}, inplace=True)
# df = df[df["Тип аккаунта"] != "OTHER"]

# Заменяем значения столбца на 1, если не NaN, и на 0, если NaN
# 1 шаг - замена на True и False
# 2 шаг - замена на 1 и 0
df['Почта существует'] = df['Найденная почта'].notna().astype(int)

# Разделяем ссылки в строках через '\n'
df['Ссылки из описания'] = df['Ссылки из описания'].str.split('\n')
df['Ссылки из контактов'] = df['Ссылки из контактов'].str.split('\n')


##################################
# Шаг 2: Делим на X и y + разделяем данные на обучающую и тестовую выборки
##################################
X = df[[
    "Описание страницы",  # text
    "Ссылки из описания",  # text
    "Ссылки из контактов",  # text
    "Почта существует",  # 1/0
    "Кол-во постов",  # num
]]
y = df["Тип аккаунта"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

##################################
# Шаг 3: Настраиваем векторизацию и классификацию + cобираем Pipeline 
##################################

# Создание ColumnTransformer для обработки каждого столбца
preprocessor = ColumnTransformer(
    transformers=[
        (
            'desc_tfidf',  # Для текстовых значений 
            Pipeline([
                ('text_cleaner', TextCleaner()),
                ('tfidf', TfidfVectorizer(stop_words='english',  # Стоп-слова
                                          ngram_range=(1, 3),  # Использование триграмм
                                          max_features=3500,  # Увеличение числа признаков
                                          sublinear_tf=True)  # Использование логарифмического масштаба для частоты терминов
                ),
            ]),
            'Описание страницы'
            
        ),
        (
            'desc_links_binarizer',
            Pipeline([
                ('binarizer', DomainBinarizer())
                # при желании можно добавить StandardScaler() —
                # но для бинарных фич это обычно не критично
            ]),
            'Ссылки из описания'
        ),
        (
            'contact_links_binarizer',
            Pipeline([
                ('binarizer', DomainBinarizer())
            ]),
            'Ссылки из контактов'
        ),
        (
            'binary', 
            'passthrough',  # Пропускает значения без изменений
            ['Почта существует']
        ),
        (
            'num_scaler', 
            StandardScaler(),  # Для числовых значений
            ['Кол-во постов']
        ),
    ],
    remainder='drop'  # Удаляет остальные столбцы, если они есть
)


##################################
# Шаг 4: Собираем Pipeline
##################################

# Использование методов ресэмплинга
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),  # Увеличение числа примеров для редких классов с помощью методов, таких как SMOTE.
    ('clf', LogisticRegression(class_weight='balanced', max_iter=1000)),
])

# ##################################
# # Шаг 5: Обучение и оценка
# ##################################

# Обучаем
pipeline.fit(X_train, y_train)

# Предсказываем
y_pred = pipeline.predict(X_test)

# Смотрим отчёт
print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

      ARTIST       0.89      0.92      0.90        36
   BEATMAKER       1.00      0.50      0.67         2
   COMMUNITY       0.78      0.95      0.86        19
       LABEL       1.00      0.60      0.75         5
      MARKET       1.00      0.67      0.80         6

    accuracy                           0.87        68
   macro avg       0.93      0.73      0.80        68
weighted avg       0.88      0.87      0.86        68



In [33]:
print(classification_report(y_test, y_pred, zero_division=0))

              precision    recall  f1-score   support

      ARTIST       0.72      0.74      0.73        31
   BEATMAKER       1.00      1.00      1.00         3
   COMMUNITY       0.62      0.57      0.59        14
       LABEL       0.00      0.00      0.00         2
      MARKET       0.78      0.70      0.74        10
       OTHER       0.80      0.81      0.80        81

    accuracy                           0.76       141
   macro avg       0.65      0.64      0.64       141
weighted avg       0.75      0.76      0.76       141



In [35]:
import joblib

##################################
# Шаг 6: Обучение модели на всех имеющихся данных
##################################

pipeline.fit(X, y)

##################################
# Шаг 7: Сохранение модели
##################################

model_path = 'data/modеls/account_type.pkl'
joblib.dump(pipeline, model_path)
print(f"Модель сохранена в файле {model_path}")


##################################
# Шаг 8: Загрузка модели
##################################

loaded_model = joblib.load(model_path)
print(f"Модель загружена из файла {model_path}")


##################################
# Шаг 9: Использование модели
##################################

new_data = pd.DataFrame({
    'Описание страницы': [
        '''Colt Blumenthal
mixedbycolt
🚀 #1 Billboard Engineer, 💿 Multi-Platinum
💻Engineered for 100+ Major Artists
🏅 SAE Institute Alumni Hall of Fame
🔊Premium Mixing & Mastering Service
linktr.ee/mixedbycolt'''
    ],
    'Ссылки из описания': [
        [
            'https://linktr.ee/mixedbycolt?fbclid=PAZXh0bgNhZW0CMTEAAaYAv9ZUzP17l1o8wVsSVjvTAzUJT9CppNIKLuP0FENyTVKEOAH3OmmBnCA_aem_FRFJBiIdaHkZ9aYHEaCLHQ',
            'https://www.instagram.com/explore/tags/1/',
            'https://www.threads.net/@mixedbycolt?xmt=AQGzUA43HPJ5PxWfBGKXf29C7EUpu6r6IVFcdtq1jbBJa8Y'
        ]
    ],
    'Ссылки из контактов': [
        ''
        # 'https://soundcloud.com/playlist/ddw31e\nhttps://youtube.com/watch?v=12345'
    ],
    'Почта существует': [0],
    'Кол-во постов': [90]
})

preds = loaded_model.predict(new_data)
preds_enum = [AccountType(s) for s in preds]  # [AccountType.ARTIST, AccountType.LABEL, ...]

print("Предсказанные классы:", preds_enum)

proba = loaded_model.predict_proba(new_data)
print("Вероятности классов:", proba)

Модель сохранена в файле data/modеls/account_type.pkl
Модель загружена из файла data/modеls/account_type.pkl
Предсказанные классы: [<AccountType.BEATMAKER: 'BEATMAKER'>]
Вероятности классов: [[0.06444823 0.85820526 0.02759148 0.03312428 0.01663076]]


In [39]:
str(AccountType(preds[0]))

'AccountType.BEATMAKER'

In [ ]:
import numpy as np
from robot.models import AccountType  # Импортируем Enum с типами аккаунтов
from robot import config


def get_account_type(data: pd.DataFrame, threshold: int = 0.8):
    """
    Предсказывает и возвращает тип аккаунта
    """
    loaded_model = joblib.load(config.ACCOUNT_TYPE_MODEL_PATH)
    
    preds = loaded_model.predict(data)  # Предсказанные классы
    probs = loaded_model.predict_proba(data)  # Вероятности классов

    # Индекс предсказанного класса
    pred_index = np.argmax(probs[0])  # Максимальная вероятность в первой строке

    # Проверяем вероятность
    if probs[0][pred_index] < threshold:  # Если вероятность меньше threshold (в %)
        pred = AccountType.UNKNOWN  # Назначаем тип UNKNOWN
    else:
        pred = AccountType(preds[0])  # Преобразуем строку в enum

    return pred.value


Предсказанный тип аккаунта: AccountType.BEATMAKER


In [76]:
import json

post_links = ['https://www.instagram.com/p/OLD1/', 'https://www.instagram.com/p/OLD2/']

path = 'data/in/post_links.json'
with open(path, 'w') as f:
    f.write(json.dumps(post_links))





In [77]:

# Получение старых ссылок
with open(path, 'r') as f:
    old_post_links = json.load(f)
    print(old_post_links)

# Проверка новых ссылок
new_post_links = ['https://www.instagram.com/p/NEW1/', 'https://www.instagram.com/p/NEW2/', 'https://www.instagram.com/p/OLD1/']
for link in new_post_links:
    if link not in old_post_links:
        print(f'Обрабатываю {link}')

# Запись новых ссылок в файл
with open(path, 'w') as f:
    post_links = list(set(old_post_links + new_post_links))
    print(post_links)
    f.write(json.dumps(post_links, indent=4))

['https://www.instagram.com/p/OLD1/', 'https://www.instagram.com/p/OLD2/']
Обрабатываю https://www.instagram.com/p/NEW1/
Обрабатываю https://www.instagram.com/p/NEW2/
['https://www.instagram.com/p/NEW1/', 'https://www.instagram.com/p/OLD2/', 'https://www.instagram.com/p/OLD1/', 'https://www.instagram.com/p/NEW2/']


In [ ]:
# Подсчет каждого типа аккаунта
print(df['Тип аккаунта'].value_counts())


In [ ]:
# precision (Точность) - какая доля действительно относится к этому классу?
# recall (Полнота) - какая доля была предсказана как этот класс?
# f1-score (Среднее гармоническое между точностью и полнотой)
# support (Число реальных образцов)

# accuracy: Общая доля правильных предсказаний по всем классам.

# avg: Средние значения precision, recall и f1-score по всем классам (без учёта их пропорций). 
# То есть каждый класс “взвешен” одинаково, даже если в выборке один класс встречается гораздо чаще другого.

# weighted avg: Взвешенное среднее тех же метрик, где вес — это support (количество объектов каждого класса). 
# Таким образом, частые классы влияют на метрики сильнее, чем редкие.


In [ ]:
# Фильтруем только строки, где "Предсказанный тип аккаунта" == "ARTIST"
df_artist = df[df["Предсказанный тип аккаунта"] == "ARTIST"]

# Считаем самые популярные хэштеги среди этих строк
hashtags_count = df_artist["Хэштег, по которому найден аккаунт"].value_counts()

# Смотрим результат (топ 10, например)
print(hashtags_count.head(10))


In [ ]:
duplicates_mask = df.duplicated()
print("Число дубликатов:", duplicates_mask.sum())

### Объединение данных с обработанными таблицами

In [1]:
import os
import pandas as pd
import numpy as np

from helpers.excel import write_excel

from database.orm import async_session, get_all_accounts


In [ ]:
# Папка с файлами .ods
folder_path = 'data/modеls/tables'

# Получаем список файлов .ods в папке
files = [f for f in os.listdir(folder_path) if f.endswith('.ods')]

# Создаем пустой DataFrame для объединенных данных
combined_df = pd.DataFrame()

for file in files:
    # Загружаем данные из каждого файла
    file_path = os.path.join(folder_path, file)
    data = pd.read_excel(file_path, engine='odf')  # Используем 'odf' для .ods файлов
    
    # Очистка данных
    data['Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER'] = data['Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER'].replace(np.nan, 'OTHER')

    # Объединяем данные по названию столбцов
    combined_df = pd.concat([combined_df, data], ignore_index=True)

async def update_account_types(async_session, data: pd.DataFrame):
    async with async_session() as session:
        # Получаем все аккаунты из базы данных
        accounts = await get_all_accounts(async_session)

        # Создаем словарь для быстрого поиска аккаунтов по ссылке
        account_dict = {account.link: account for account in accounts}

        # Обновляем типы аккаунтов на основе данных из DataFrame
        for _, row in data.iterrows():
            link = row.get('Ссылка на аккаунт').strip()
            account_type = row.get('Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER').strip()
            # print(f'link: {link}')
            # print(f'account_type: {account_type}')

            if link in account_dict and account_type:
                account = account_dict[link]
                account.account_type = account_type
                # print(f'account: {account.link}')
                # print(f'account_type: {account.account_type}')
                session.add(account)
        # Сохраняем изменения в базе данных
        await session.commit()
        
await update_account_types(async_session, combined_df)

In [ ]:
accounts = await get_all_accounts(async_session)
write_excel(accounts, out_path='data/modеls/row_data/merged_table.xlsx')